In [22]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Dropout, Flatten
from keras import backend
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [27]:
#Pandas DataFrame
dataset = pd.read_csv("heston_training.csv")
dataset = dataset.dropna()

X = dataset.iloc[:, :9]
# l'ho trasformato in np.array per comodita`
y = dataset.iloc[:, 9].to_numpy()

#### Se la normalizzazione la fai prima di splittare in training e test fai prima...
Con il rescale dell'output ricordati che l'errore deve essere reinterpretato, nel senso che se il prezzo e' nel range 50-150 riscalato diventa 0-1, ma a questo punto l'errore 1e-3 che senza rescale era ottimo, adesso ti serve 1e-5...

**ah metti un header nel csv con in nomi dei parametri cosi' poi puoi chiamare le colonne anche per "nome"**

In [28]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

print (np.min(X), np.max(X))
scale_x = MinMaxScaler().fit(X)
X = scale_x.transform(X)
print (np.min(X), np.max(X))

# per fare il rescale dell'output devi trasformare l'array in bidimensionale
# cosi se lo aspetta la funzione
y = y.reshape(len(y), 1)
print (np.min(y), np.max(y))
scale_y = MinMaxScaler().fit(y)
y = scale_y.transform(y)
print (np.min(y), np.max(y))

# ti conviene salvarti le trasformazioni cosi`
import joblib
joblib.dump(scale_x, "heston_x_scaler.save")
joblib.dump(scale_y, "heston_y_scaler.save")

# se ti serve puoi ricaricare cosi`
#scale_x = joblib.load("heston_x_scaler.save")
#scale_y = joblib.load("heston_y_scaler.save")

p1      2.00
p2      0.05
p3      0.10
p4     -0.50
p5      0.08
p6      0.05
p7      0.50
p8    115.00
p9     80.00
dtype: float64 p1      2.00
p2      0.06
p3      0.55
p4     -0.35
p5      0.08
p6      0.05
p7      2.00
p8    115.00
p9    120.00
dtype: float64
0.0 1.0
3.9327372005205774 43.2865920175541
0.0 1.0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(151641, 9) (151641, 1)
(37911, 9) (37911, 1)


In [30]:
#create model

model = Sequential()
model.add(Dense(40, input_dim=9, kernel_initializer='normal', activation='tanh'))
model.add(Dense(20, kernel_initializer='normal', activation='tanh'))
model.add(Dense(5, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))

In [31]:
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse', 'mae'])

In [32]:
history = model.fit(X_train, y_train, epochs=10, verbose=1)#, batch_size=10) #epochs = 1000

Epoch 1/10
 68768/151641 [============>.................] - ETA: 13s - loss: 0.0374 - mse: 0.0099 - mae: 0.0374

KeyboardInterrupt: 

In [52]:
evaluator = model.evaluate(X_train, y_train)
print('Test: {}'.format(evaluator))


151640/151640 [==============================] - 2s 16us/step
Test: [0.040797139905674164, 0.0034732921048998833, 0.04079712927341461]


In [53]:
evaluator = model.evaluate(X_test, y_test)
print('Test: {}'.format(evaluator))


37911/37911 [==============================] - 1s 14us/step
Test: [0.04092981137370872, 0.003456472884863615, 0.040929798036813736]


In [54]:
model.save('heston_model.h5')